In [26]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import pickle
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
eval_results_a = "/Users/davidkorcak/Documents/ctu/bachelors/flipper_training/cross_eval_results/final_mixed_objective_training_666_policy_final/"
eval_results_a = Path(eval_results_a)
eval_results_b = "/Users/davidkorcak/Documents/ctu/bachelors/flipper_training/cross_eval_results/mixed_training_smoothness_policy_final"
eval_results_b = Path(eval_results_b)

In [28]:
name = f"{eval_results_a.stem}_vs_{eval_results_b.stem}"

In [29]:
results_dict_a = {}
for f in eval_results_a.glob("*.pkl"):
    with open(f, "rb") as r:
        results = pickle.load(r)  # noqa
    results_dict_a[f.stem] = results

results_dict_b = {}
for f in eval_results_b.glob("*.pkl"):
    with open(f, "rb") as r:
        results = pickle.load(r)  # noqa
    results_dict_b[f.stem] = results

In [30]:
def list_of_dicts_to_dict_of_lists(list_of_dicts):
    """
    Convert a list of dictionaries to a dictionary of lists.
    """
    dict_of_lists = {}
    for d in list_of_dicts:
        for k, v in d.items():
            if k not in dict_of_lists:
                dict_of_lists[k] = []
            dict_of_lists[k].append(v)
    return dict_of_lists

In [31]:
key2name = {
    "training": "Training Dist",
    "stairs_hard": "Stairs-Hard",
    "stairs_easy": "Stairs-Easy",
    "barrier_easy": "Barrier-Easy",
    "barrier_hard": "Barrier-Hard",
    "gauss_coarse_hard": "Gauss-Coarse-Hard",
    "gauss_fine_easy": "Gauss-Fine-Easy",
    "gauss_fine_hard": "Gauss-Fine-Hard",
    "gauss_coarse_easy": "Gauss-Coarse-Easy",
    "trunk_easy": "Trunk-Easy",
    "trunk_hard": "Trunk-Hard",
}

In [32]:
results_transposed_a = {k: list_of_dicts_to_dict_of_lists(v) for k, v in results_dict_a.items()}
results_transposed_b = {k: list_of_dicts_to_dict_of_lists(v) for k, v in results_dict_b.items()}

In [39]:
r1 = results_transposed_b
r2 = results_transposed_a
results_transposed_diff = {}
for k in r1.keys():
    if k not in r2:
        continue
    results_transposed_diff[k] = {}
    for k2 in r1[k].keys():
        if k2 not in r2[k]:
            continue
        results_transposed_diff[k][k2] = []
        for i in range(len(r1[k][k2])):
            results_transposed_diff[k][k2].append(r1[k][k2][i] - r2[k][k2][i])


results_transposed_diff

{'stairs_easy': {'eval/mean_step_reward': [-0.5292067527770996,
   0.6250135898590088,
   -0.6273713111877441,
   -0.28565454483032227,
   -0.5085020065307617,
   1.337463617324829,
   1.1856579780578613,
   -2.028655767440796,
   -0.06146812438964844,
   -0.3706216812133789,
   2.882319211959839,
   1.5478615760803223,
   0.10631752014160156,
   -1.169644832611084,
   -0.44653940200805664,
   -0.6991534233093262,
   -0.12143182754516602,
   -0.3822154998779297,
   1.7932316660881042,
   -0.08295536041259766,
   -2.1779911518096924,
   1.2244079113006592,
   -1.0386583805084229,
   -0.9224624633789062,
   -2.140530824661255,
   -0.1207280158996582,
   1.3434479236602783,
   0.14705276489257812,
   0.9919719696044922,
   1.6592206954956055],
  'eval/max_step_reward': [-0.15494346618652344,
   -0.6434221267700195,
   -0.4504871368408203,
   0.04183006286621094,
   -0.40253734588623047,
   -0.28997230529785156,
   0.7561531066894531,
   -0.3775672912597656,
   0.27135753631591797,
   -2.0

In [40]:
results_transposed_a["gauss_fine_easy"]["eval/pct_succeeded"], results_transposed_b["gauss_fine_easy"]["eval/pct_succeeded"]

([0.9375,
  0.75,
  0.875,
  0.75,
  0.875,
  0.6875,
  0.8125,
  0.875,
  0.6875,
  0.6875,
  0.8125,
  0.875,
  0.875,
  0.8125,
  1.0,
  0.8125,
  0.8125,
  0.75,
  0.625,
  0.8125,
  0.9375,
  0.75,
  0.5,
  0.625,
  0.75,
  0.8125,
  0.6875,
  0.8125,
  0.9375,
  0.6875],
 [0.875,
  0.4375,
  0.5,
  0.625,
  0.5625,
  0.4375,
  0.5,
  0.6875,
  0.3125,
  0.5,
  0.5,
  0.4375,
  0.625,
  0.4375,
  0.6875,
  0.625,
  0.4375,
  0.5625,
  0.625,
  0.3125,
  0.6875,
  0.8125,
  0.0625,
  0.6875,
  0.5625,
  0.5625,
  0.625,
  0.5,
  0.75,
  0.125])

In [41]:
easy_keys = ["training"] + [k for k in results_transposed_diff if "easy" in k]
hard_keys = ["training"] + [k for k in results_transposed_diff if "hard" in k]
easy_label_keys = [key2name[k] for k in easy_keys] + ["All-Easy"]
hard_label_keys = [key2name[k] for k in hard_keys] + ["All-Hard"]

In [43]:
import numpy as np
from pathlib import Path

# Define a directory to save the .tex table files
tables_output_dir = Path("/Users/davidkorcak/Documents/ctu/bachelors/bachelor_thesis") / "tables"
os.makedirs(tables_output_dir, exist_ok=True)


def compute_dual_metric_stats(
    env_data_keys_to_process,  # List of raw keys: e.g., ["training", "stairs_easy"] (without "All")
    full_results_data,  # results_transposed
    environment_name_mapping,  # key2name
):
    """
    Computes mean and standard deviation for success and failure rates across specified environments.
    Also calculates an "All" aggregate for these environments.
    Returns a dictionary structured as: {env_display_name: {"success": (mean, std), "failure": (mean, std)}, ...}
    The "All" key in the returned dictionary contains aggregated stats.
    """
    stats_by_env_then_metric = {}  # {display_name: {"success": (m,s), "failure": (m,s)}, ...}

    overall_success_values = []
    overall_failure_values = []

    for env_key in env_data_keys_to_process:  # Iterate through specific environments first
        display_name = environment_name_mapping.get(env_key)
        if display_name is None:
            print(f"Warning: No display name found for environment key '{env_key}' in key2name. Skipping.")
            stats_by_env_then_metric[env_key] = {"success": (np.nan, np.nan), "failure": (np.nan, np.nan)}  # Store with raw key if no display name
            continue

        stats_by_env_then_metric[display_name] = {}

        # Success metric
        if env_key in full_results_data and "eval/pct_succeeded" in full_results_data[env_key]:
            success_values = full_results_data[env_key]["eval/pct_succeeded"]
            if success_values:  # Ensure list is not empty
                mean_s = np.mean(success_values) * 100
                std_s = np.std(success_values) * 100
                stats_by_env_then_metric[display_name]["success"] = (mean_s, std_s)
                if env_key != "training":  # Exclude training from overall stats
                    overall_success_values.extend(success_values)  # Add to overall list for "All" calculation
            else:
                stats_by_env_then_metric[display_name]["success"] = (np.nan, np.nan)
        else:
            print(f"Warning: Success metric 'eval/pct_succeeded' not found for env '{env_key}'. Stats for '{display_name}' success will be NaN.")
            stats_by_env_then_metric[display_name]["success"] = (np.nan, np.nan)

        # Failure metric
        if env_key in full_results_data and "eval/pct_failed" in full_results_data[env_key]:
            failure_values = full_results_data[env_key]["eval/pct_failed"]
            if failure_values:  # Ensure list is not empty
                mean_f = np.mean(failure_values) * 100
                std_f = np.std(failure_values) * 100
                stats_by_env_then_metric[display_name]["failure"] = (mean_f, std_f)
                if env_key != "training":  # Exclude training from overall stats
                    overall_failure_values.extend(failure_values)  # Add to overall list for "All" calculation
            else:
                stats_by_env_then_metric[display_name]["failure"] = (np.nan, np.nan)
        else:
            print(f"Warning: Failure metric 'eval/pct_failed' not found for env '{env_key}'. Stats for '{display_name}' failure will be NaN.")
            stats_by_env_then_metric[display_name]["failure"] = (np.nan, np.nan)

    # Calculate "All" category stats from all collected values
    all_key = "All-" + ("Easy" if any("easy" in k for k in env_data_keys_to_process) else "Hard")
    stats_by_env_then_metric[all_key] = {}
    if overall_success_values:
        mean_all_s = np.mean(overall_success_values) * 100
        std_all_s = np.std(overall_success_values) * 100
        stats_by_env_then_metric[all_key]["success"] = (mean_all_s, std_all_s)
    else:
        print("Warning: No data aggregated for 'All' success category. Stats will be NaN.")
        stats_by_env_then_metric[all_key]["success"] = (np.nan, np.nan)

    if overall_failure_values:
        mean_all_f = np.mean(overall_failure_values) * 100
        std_all_f = np.std(overall_failure_values) * 100
        stats_by_env_then_metric[all_key]["failure"] = (mean_all_f, std_all_f)
    else:
        print("Warning: No data aggregated for 'All' failure category. Stats will be NaN.")
        stats_by_env_then_metric[all_key]["failure"] = (np.nan, np.nan)

    return stats_by_env_then_metric


def create_latex_table_new_format(
    stats_per_env,  # Dict from compute_dual_metric_stats: {env_display_name: {"success": (m,s), "failure": (m,s)}, ...}
    ordered_env_display_names,  # List of display names in desired row order, e.g., ["Training Dist", ..., "All"]
    caption_text,
    label_text,
    font_size="\\footnotesize",
):
    """
    Generates a LaTeX table string with environments as rows and MSR/MFR as columns.
    """
    latex_code = "\\begin{table}[H]\n"
    latex_code += "\\centering\n"
    latex_code += f"{font_size}\n"
    # 'l' for environment name (left-aligned), 'c' for MSR, 'c' for MFR (center-aligned)
    latex_code += "\\begin{tabular}{lcc}\n"
    latex_code += "\\toprule\n"
    latex_code += "Environment & Success Rate Difference (\\%) $\\pm 1 \\sigma$ & Failure Rate Difference (\\%) $\\pm 1 \\sigma$ \\\\\n"
    latex_code += "\\midrule\n"

    for env_display_name in ordered_env_display_names:
        env_metrics = stats_per_env.get(env_display_name, {})  # Get stats for the current display name

        success_data = env_metrics.get("success", (np.nan, np.nan))
        failure_data = env_metrics.get("failure", (np.nan, np.nan))

        msr_entry = "N/A"
        if not np.isnan(success_data[0]) and not np.isnan(success_data[1]):
            msr_entry = f"${success_data[0]:.2f} \\pm {success_data[1]:.2f}$"

        mfr_entry = "N/A"
        if not np.isnan(failure_data[0]) and not np.isnan(failure_data[1]):
            mfr_entry = f"${failure_data[0]:.2f} \\pm {failure_data[1]:.2f}$"

        latex_code += f"{env_display_name} & {msr_entry} & {mfr_entry} \\\\\n"

    latex_code += "\\bottomrule\n"
    latex_code += "\\end{tabular}\n"
    latex_code += f"\\caption{{{caption_text}}}\n"
    latex_code += f"\\label{{{label_text}}}\n"
    latex_code += "\\end{table}\n"

    return latex_code


# --- Configuration for the two tables (Easy and Hard) ---
# easy_keys, hard_keys, easy_label_keys, hard_label_keys are defined in cell 7.
# easy_keys/hard_keys do NOT include "All" - this is handled by compute_dual_metric_stats.
# easy_label_keys/hard_label_keys DO include "All" at the end, which is used for ordering rows.

table_configurations_new = [
    {
        "type": "Easy Environments",
        # env_data_keys_to_process should not include "All" as compute_dual_metric_stats calculates it.
        "env_data_keys_to_process": easy_keys,
        # ordered_env_display_names includes "All" for row ordering.
        "ordered_env_display_names": easy_label_keys,
        "caption": "Success and failure rate difference statistics for \\textit{easy} environments and training distribution aggregated over multiple evaluation seeds.",
        "label_suffix": "_easy_metrics",
        "filename_suffix": "_easy_metrics.tex",
    },
    {
        "type": "Hard Environments",
        "env_data_keys_to_process": hard_keys,
        "ordered_env_display_names": hard_label_keys,
        "caption": "Success and failure rate difference statistics for \\textit{hard} environments and training distribution aggregated over multiple evaluation seeds.",
        "label_suffix": "_hard_metrics",
        "filename_suffix": "_hard_metrics.tex",
    },
]

# --- Generate and save/print each table using the new format ---
for config in table_configurations_new:
    print(f"--- Generating table for: {config['type']} ---")

    # 1. Compute statistics for both success and failure rates
    # results_transposed and key2name are from earlier cells
    current_env_stats = compute_dual_metric_stats(
        env_data_keys_to_process=config["env_data_keys_to_process"],
        full_results_data=results_transposed_diff,
        environment_name_mapping=key2name,
    )

    # 2. Create LaTeX table string
    # eval_results_dir.stem is from an earlier cell
    table_label_str = f"tab:{name}{config['label_suffix']}"
    latex_output_str = create_latex_table_new_format(
        stats_per_env=current_env_stats,
        ordered_env_display_names=config["ordered_env_display_names"],
        caption_text=config["caption"],
        label_text=table_label_str,
    )

    # 3. Save to .tex file
    output_file_path_str = tables_output_dir / f"{name}{config['filename_suffix']}"
    with open(output_file_path_str, "w") as f_out:
        f_out.write(latex_output_str)
    print(f"Saved {config['type']} table to: {output_file_path_str}\n")

    # 4. Print LaTeX to console
    print(latex_output_str)
    print("-" * 50 + "\n")

print(f"All new format LaTeX tables have been generated and saved in: {tables_output_dir}")

--- Generating table for: Easy Environments ---
Saved Easy Environments table to: /Users/davidkorcak/Documents/ctu/bachelors/bachelor_thesis/tables/final_mixed_objective_training_666_policy_final_vs_mixed_training_smoothness_policy_final_easy_metrics.tex

\begin{table}[H]
\centering
\footnotesize
\begin{tabular}{lcc}
\toprule
Environment & Success Rate Difference (\%) $\pm 1 \sigma$ & Failure Rate Difference (\%) $\pm 1 \sigma$ \\
\midrule
Training Dist & $-5.62 \pm 16.56$ & $-5.62 \pm 6.32$ \\
Stairs-Easy & $5.83 \pm 14.25$ & $-1.46 \pm 3.49$ \\
Barrier-Easy & $-2.92 \pm 5.03$ & $-0.62 \pm 4.38$ \\
Gauss-Coarse-Easy & $13.75 \pm 13.73$ & $-3.12 \pm 6.80$ \\
Gauss-Fine-Easy & $-25.21 \pm 15.09$ & $0.21 \pm 1.97$ \\
Trunk-Easy & $-0.83 \pm 10.55$ & $0.00 \pm 1.61$ \\
All-Easy & $-1.88 \pm 17.92$ & $-1.00 \pm 4.27$ \\
\bottomrule
\end{tabular}
\caption{Success and failure rate difference statistics for \textit{easy} environments and training distribution aggregated over multiple evaluati